In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scikit_posthocs as sp
import scipy.stats as stats
from sklearn import preprocessing
from sklearn.metrics import r2_score

from statsmodels.stats.multicomp import MultiComparison
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import metrics


from scipy.stats import linregress

### Lendo o csv

In [2]:
df = pd.read_csv('datasets/banco_por_semestre.csv', sep = ',', encoding = "ISO-8859-1", low_memory = False, error_bad_lines=False).drop(['Unnamed: 0'], axis=1,errors='ignore')
df.head()

,Id,SEMESTRE,DN,Idade,Codsexo,Raça,pesoi,pesof,Alt,instruc,...,TFG,TFG_EQ,Estagio,Estagio_EQ,ESTAGIOI,ESTAGIOF,ESTAGIOI_EQ,ESTAGIOF_EQ,CREATININAI,CREATININAF
0,27,2010,29-May-40,74.0,Masculino,Branca,74.9,70.0,167.0,Fundamental completo,...,NaN,NaN,NaN,NaN,Estágio 3a - 45-59 ml,Estágio 2 - 60-89 ml,Estágio 3a - 45-59 ml,Estágio 2 - 60-89 ml,1.4,1.1
1,27,2011-1,29-May-40,74.0,Masculino,Branca,74.9,70.0,167.0,Fundamental completo,...,52.43,52.65,Estágio 3a - 45-59 ml,Estágio 3a - 45-59 ml,Estágio 3a - 45-59 ml,Estágio 2 - 60-89 ml,Estágio 3a - 45-59 ml,Estágio 2 - 60-89 ml,1.4,1.1
2,27,2011-2,29-May-40,74.0,Masculino,Branca,74.9,70.0,167.0,Fundamental completo,...,60.31,60.57,Estágio 2 - 60-89 ml,Estágio 2 - 60-89 ml,Estágio 3a - 45-59 ml,Estágio 2 - 60-89 ml,Estágio 3a - 45-59 ml,Estágio 2 - 60-89 ml,1.4,1.1
3,27,2012-1,29-May-40,74.0,Masculino,Branca,74.9,70.0,167.0,Fundamental completo,...,NaN,NaN,NaN,NaN,Estágio 3a - 45-59 ml,Estágio 2 - 60-89 ml,Estágio 3a - 45-59 ml,Estágio 2 - 60-89 ml,1.4,1.1
4,27,2012-2,29-May-40,74.0,Masculino,Branca,74.9,70.0,167.0,Fundamental completo,...,44.94,45.13,Estágio 3b - 30-44 ml,Estágio 3a - 45-59 ml,Estágio 3a - 45-59 ml,Estágio 2 - 60-89 ml,Estágio 3a - 45-59 ml,Estágio 2 - 60-89 ml,1.4,1.1


In [3]:
# passandoa lgumas colunas mais importantes para inteiro
df['Raça'].replace(['Branca','Parda','Preta','Indigena','Amarela'],[1,2,3,4,5], inplace=True)
df['tabagismo'].replace(['Ex','Sim'],[0,1], inplace=True)
df['Codsexo'].replace(['Feminino','Masculino'],[1,2], inplace=True)

#ao invés de mexermos com o estágio final, iremos analisar o estágio do semstre
# df["SEMESTRE"].replace(df["SEMESTRE"].unique(),[2010.0, 2011.1, 2011.2, 2012.1, 2012.2, 2013.1, 2013.2,
#        2014.1, 2014.2, 2015.0], inplace=True)
df["Estagio_EQ"].replace(df["Estagio_EQ"].unique()[1:],[3,2,5,4,1,6], inplace=True)

df['etilismo'].replace(['Ex','Sim'],[0,1], inplace=True)
df['sedentario'].replace(['Não','Sim'],[0,1], inplace=True)
df['insulina'].replace(['Não','Sim'],[0,1], inplace=True)

### Exames mais importantes

In [4]:
exames =['Codsexo', 'Idade', 'Raça', 'PAS','PAD' , 
           'pesoi', 'pesof', 'Hemoglobina', 
           'ColesterolTotal', 'GlicemiadeJejum', 'Triglicerides', 
           'Potassio', 'ColesterolHDL', 'Ureia', 'TSH',
           'AcidoUrico', 'HemoglobinaGlicada', 'TGP',
           'sedentario', 
           'etilismo', 'tabagismo','Estagio_EQ']
#colcocar em um dataframe
df_est= pd.DataFrame(index=[i for i in range(len(df))], columns = exames)
for i in exames:
    df_est[i] = df[i]
df_est

,Codsexo,Idade,Raça,PAS,PAD,pesoi,pesof,Hemoglobina,ColesterolTotal,GlicemiadeJejum,...,ColesterolHDL,Ureia,TSH,AcidoUrico,HemoglobinaGlicada,TGP,sedentario,etilismo,tabagismo,Estagio_EQ
0,2,74.0,1,137.0,78.0,74.9,70.0,16.2,216.0,196.0,...,47.0,NaN,NaN,NaN,10.4,NaN,1.0,NaN,0.0,NaN
1,2,74.0,1,NaN,NaN,74.9,70.0,NaN,NaN,NaN,...,NaN,NaN,0.96,5.9,NaN,19.0,1.0,NaN,0.0,3.0
2,2,74.0,1,NaN,NaN,74.9,70.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0.0,2.0
3,2,74.0,1,NaN,NaN,74.9,70.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0.0,NaN
4,2,74.0,1,NaN,NaN,74.9,70.0,NaN,NaN,NaN,...,NaN,46.0,NaN,NaN,NaN,NaN,1.0,NaN,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56885,1,60.0,3,NaN,NaN,102.0,102.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56886,1,60.0,3,NaN,NaN,102.0,102.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56887,1,60.0,3,NaN,NaN,102.0,102.0,NaN,208.0,NaN,...,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
56888,1,60.0,3,136.0,80.0,102.0,102.0,13.5,191.0,NaN,...,39.0,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,2.0


### Separando por estagios

In [5]:
df_estagio=[]

df_estagio.append([i for i in df_est.groupby('Estagio_EQ')][0][1])#1
df_estagio.append([i for i in df_est.groupby('Estagio_EQ')][1][1])#2
df_estagio.append([i for i in df_est.groupby('Estagio_EQ')][2][1])#3a
df_estagio.append([i for i in df_est.groupby('Estagio_EQ')][3][1])#3b
df_estagio.append([i for i in df_est.groupby('Estagio_EQ')][4][1])#4
df_estagio.append([i for i in df_est.groupby('Estagio_EQ')][5][1])#5

### Fazer retirando as linhas que possuem nan no exame

Nesta parte estarei retirando todas linhas que não possuem valores do exame correspondente, fazendo a correlação de cada exame com o seu respestivo valor de Estagio_EQ.

In [27]:
array_corr =[]
df_corr=pd.DataFrame(index=df_est.keys(),columns=['Estagio_EQ_1','Estagio_EQ_2','Estagio_EQ_3a',
                                                       'Estagio_EQ_3b','Estagio_EQ_4','Estagio_EQ_5'])
nomes=['Estagio_EQ_1','Estagio_EQ_2','Estagio_EQ_3a',
                                                       'Estagio_EQ_3b','Estagio_EQ_4','Estagio_EQ_5']
for i in range(len(df_estagio)):
#     print(i)
#     df_corr=pd.DataFrame(index=estagio.keys(),columns=['Estagio_EQ'])
    for exame in df_estagio[i]:
        #montando um datframe só com as linhas que possuem valores do exame específico
        df_aux = df_est.dropna(subset=[exame])
        print('tamanho:'+str(df_aux.shape))
#         print(df_aux)
#         print(df_aux.shape)
        corr_exame = df_aux[exame].corr(df_aux['Estagio_EQ'])
#         if exame=='Hemoglobina':
#             print(df_aux)
#             print(df_aux['Hemoglobina'].isnull().any())
#             print(corr_exame)
        df_corr[nomes[i]].loc[exame] = corr_exame
    print('-------------------')
#         print(df_corr)
array_corr.append(df_corr)


tamanho:(56890, 22)
tamanho:(56890, 22)
tamanho:(56890, 22)
tamanho:(11336, 22)
tamanho:(11336, 22)
tamanho:(56390, 22)
tamanho:(56390, 22)
tamanho:(9321, 22)
tamanho:(9472, 22)
tamanho:(9588, 22)
tamanho:(9324, 22)
tamanho:(8154, 22)
tamanho:(9225, 22)
tamanho:(7311, 22)
tamanho:(7049, 22)
tamanho:(7026, 22)
tamanho:(7426, 22)
tamanho:(6945, 22)
tamanho:(44480, 22)
tamanho:(16590, 22)
tamanho:(22550, 22)
tamanho:(14707, 22)
-------------------
tamanho:(56890, 22)
tamanho:(56890, 22)
tamanho:(56890, 22)
tamanho:(11336, 22)
tamanho:(11336, 22)
tamanho:(56390, 22)
tamanho:(56390, 22)
tamanho:(9321, 22)
tamanho:(9472, 22)
tamanho:(9588, 22)
tamanho:(9324, 22)
tamanho:(8154, 22)
tamanho:(9225, 22)
tamanho:(7311, 22)
tamanho:(7049, 22)
tamanho:(7026, 22)
tamanho:(7426, 22)
tamanho:(6945, 22)
tamanho:(44480, 22)
tamanho:(16590, 22)
tamanho:(22550, 22)
tamanho:(14707, 22)
-------------------
tamanho:(56890, 22)
tamanho:(56890, 22)
tamanho:(56890, 22)
tamanho:(11336, 22)
tamanho:(11336, 22)
ta

In [28]:
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s>0.6
    return ['background-color: yellow}' if v else '' for v in is_max]

In [29]:
array_corr[0].style.apply(highlight_max)

,Estagio_EQ_1,Estagio_EQ_2,Estagio_EQ_3a,Estagio_EQ_3b,Estagio_EQ_4,Estagio_EQ_5
Codsexo,0.008061,0.008061,0.008061,0.008061,0.008061,0.008061
Idade,0.394378,0.394378,0.394378,0.394378,0.394378,0.394378
Raça,-0.092061,-0.092061,-0.092061,-0.092061,-0.092061,-0.092061
PAS,0.069517,0.069517,0.069517,0.069517,0.069517,0.069517
PAD,-0.007724,-0.007724,-0.007724,-0.007724,-0.007724,-0.007724
pesoi,-0.143812,-0.143812,-0.143812,-0.143812,-0.143812,-0.143812
pesof,-0.166041,-0.166041,-0.166041,-0.166041,-0.166041,-0.166041
Hemoglobina,-0.314683,-0.314683,-0.314683,-0.314683,-0.314683,-0.314683
ColesterolTotal,-0.018894,-0.018894,-0.018894,-0.018894,-0.018894,-0.018894
GlicemiadeJejum,-0.163246,-0.163246,-0.163246,-0.163246,-0.163246,-0.163246


In [18]:
df_correlacao_por_estagio = array_corr[0].style.apply(highlight_max)
df_correlacao_por_estagio.to_excel('correlacao_estagio.xlsx')

In [15]:
# array_corr[0].style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'),axis=0)
# array_corr[0].style.apply(highlight_max)

In [102]:
# array_corr[1].style.apply(highlight_max)

,Estagio_EQ
Codsexo,0.008061
Idade,0.394378
Raça,-0.092061
PAS,0.069517
PAD,-0.007724
pesoi,-0.143812
pesof,-0.166041
Hemoglobina,-0.314683
ColesterolTotal,-0.018894
GlicemiadeJejum,-0.163246


In [40]:
# array_corr[2] = array_corr[2].style.apply(highlight_max)

In [41]:
# array_corr[3] = array_corr[3].style.apply(highlight_max)

In [42]:
# array_corr[4] = array_corr[4].style.apply(highlight_max)

In [43]:
# array_corr[5] = array_corr[5].style.apply(highlight_max)

In [44]:
corr_name=['1','2','3a','3b','4','5']
cont=0
for corr in array_corr:
    corr.to_excel('correlacao_estagio_'+corr_name[cont]+'.xlsx')
    cont+=1